Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
print(dir(tf))
graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

['AggregationMethod', 'Assert', 'AttrValue', 'COMPILER_VERSION', 'ConditionalAccumulator', 'ConditionalAccumulatorBase', 'ConfigProto', 'DType', 'DeviceSpec', 'Dimension', 'Event', 'FIFOQueue', 'FixedLenFeature', 'FixedLenSequenceFeature', 'FixedLengthRecordReader', 'GIT_VERSION', 'GPUOptions', 'GRAPH_DEF_VERSION', 'GRAPH_DEF_VERSION_MIN_CONSUMER', 'GRAPH_DEF_VERSION_MIN_PRODUCER', 'Graph', 'GraphDef', 'GraphKeys', 'GraphOptions', 'HistogramProto', 'IdentityReader', 'IndexedSlices', 'InteractiveSession', 'LogMessage', 'MetaGraphDef', 'NameAttrList', 'NoGradient', 'NodeDef', 'NotDifferentiable', 'OpError', 'Operation', 'OptimizerOptions', 'PaddingFIFOQueue', 'Print', 'PriorityQueue', 'QUANTIZED_DTYPES', 'QueueBase', 'RandomShuffleQueue', 'ReaderBase', 'RegisterGradient', 'RunMetadata', 'RunOptions', 'Session', 'SessionLog', 'SparseConditionalAccumulator', 'SparseFeature', 'SparseTensor', 'SparseTensorValue', 'Summary', 'TFRecordReader', 'Tensor', 'TensorArray', 'TensorInfo', 'TensorShap

Let's run this computation and iterate:

In [5]:
num_steps = 801


def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step % 100 == 0:
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


Loss at step 0: 18.122118
Training accuracy: 10.2%
Validation accuracy: 12.4%


Loss at step 100: 2.246418
Training accuracy: 72.8%
Validation accuracy: 70.7%


Loss at step 200: 1.803650
Training accuracy: 75.8%
Validation accuracy: 73.3%


Loss at step 300: 1.571345
Training accuracy: 77.1%
Validation accuracy: 74.1%


Loss at step 400: 1.414226
Training accuracy: 77.9%
Validation accuracy: 74.7%


Loss at step 500: 1.296120
Training accuracy: 78.5%
Validation accuracy: 74.9%


Loss at step 600: 1.202308
Training accuracy: 78.9%
Validation accuracy: 75.2%


Loss at step 700: 1.125242
Training accuracy: 79.3%
Validation accuracy: 75.4%


Loss at step 800: 1.060595
Training accuracy: 80.0%
Validation accuracy: 75.4%
Test accuracy: 82.6%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [31]:
batch_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [32]:
num_steps = 9001

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.598434
Minibatch accuracy: 9.4%
Validation accuracy: 13.0%


Minibatch loss at step 500: 1.544012
Minibatch accuracy: 74.0%
Validation accuracy: 75.3%


Minibatch loss at step 1000: 1.220595
Minibatch accuracy: 77.6%
Validation accuracy: 76.5%


Minibatch loss at step 1500: 0.967065
Minibatch accuracy: 80.3%
Validation accuracy: 77.3%


Minibatch loss at step 2000: 0.967485
Minibatch accuracy: 80.6%
Validation accuracy: 78.0%


...

Minibatch loss at step 7000: 0.738063
Minibatch accuracy: 80.6%
Validation accuracy: 80.8%


Minibatch loss at step 7500: 0.677930
Minibatch accuracy: 81.4%
Validation accuracy: 80.8%


Minibatch loss at step 8000: 0.677350
Minibatch accuracy: 82.2%
Validation accuracy: 81.2%


Minibatch loss at step 8500: 0.624194
Minibatch accuracy: 83.8%
Validation accuracy: 81.1%


Minibatch loss at step 9000: 0.747199
Minibatch accuracy: 80.6%
Validation accuracy: 81.2%
Test accuracy: 88.4%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [37]:
batch_size = 124
num_steps = 20000
graph = tf.Graph()


def fc_layer(layer_name, input_tensor,input_shape, nodes):
    with tf.name_scope(layer_name):
        weights = tf.Variable(tf.truncated_normal([input_shape, nodes]), name="W")
        biases = tf.zeros([nodes], name="B")
        logits = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("activation", logits)
        return weights, biases, logits

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size), name="train_dataset")
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name="train_labels")
    tf_valid_dataset = tf.constant(valid_dataset, name="valid_dataset")
    tf_test_dataset = tf.constant(test_dataset, name="test_dataset")

    weights_1, biases_1, hidden_layer_1 = fc_layer("HL_1", tf_train_dataset, image_size * image_size, 1042)
    hidden_layer_1_activation = tf.nn.relu(hidden_layer_1)
    weights_2, biases_2, output_layer = fc_layer("Output", hidden_layer_1_activation, 1042, num_labels)
    
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_layer))
        tf.summary.scalar("loss", loss)
    
    with tf.name_scope("optimizer"):
        optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

    train_prediction = tf.nn.softmax(output_layer)

    with tf.name_scope("valid"):
        valid_hidden_layer_in = tf.matmul(tf_valid_dataset, weights_1) + biases_1
        valid_hidden_layer_out = tf.nn.relu(valid_hidden_layer_in)
        valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer_out, weights_2) + biases_2)

    with tf.name_scope("test"):
        test_hidden_layer_in = tf.matmul(tf_test_dataset, weights_1) + biases_1
        test_hidden_layer_out = tf.nn.relu(test_hidden_layer_in)
        test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer_out, weights_2) + biases_2)

In [38]:
with tf.Session(graph=graph) as session:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("tensorboard/2")
    writer.add_graph(session.graph)
    tf.global_variables_initializer().run()
    print("Initialiszed")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        if step % 500 == 0:
            s = session.run(merged_summary, feed_dict=feed_dict)
            writer.add_summary(s, step)
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 466.565430
Minibatch accuracy: 11.3%
Validation accuracy: 30.0%


Minibatch loss at step 500: 12.920561
Minibatch accuracy: 80.6%
Validation accuracy: 79.6%


Minibatch loss at step 1000: 25.607567
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%


Minibatch loss at step 1500: 15.080732
Minibatch accuracy: 79.0%
Validation accuracy: 82.7%


Minibatch loss at step 2000: 6.916016
Minibatch accuracy: 83.1%
Validation accuracy: 83.7%


...

Minibatch loss at step 18000: 1.976082
Minibatch accuracy: 90.3%
Validation accuracy: 84.6%


Minibatch loss at step 18500: 5.422221
Minibatch accuracy: 85.5%
Validation accuracy: 84.8%


Minibatch loss at step 19000: 2.552696
Minibatch accuracy: 84.7%
Validation accuracy: 85.5%


Minibatch loss at step 19500: 2.628351
Minibatch accuracy: 90.3%
Validation accuracy: 85.2%


Test accuracy: 92.0%
